In [ ]:
import pandas as pd
from langdetect import detect

# STEP 1: Load the dataset with proper encoding
df = pd.read_csv("1.csv", encoding='ISO-8859-1')

# STEP 2: Rename columns for easier access
df.rename(columns={
    'ItemNo.': 'item_no',
    'Author': 'author',
    'Title': 'title',
    'Edition': 'edition',
    'Imprint': 'imprint',
    'Date1': 'date',
    'Call No.': 'call_no'
}, inplace=True)

# STEP 3: Drop rows missing essential info (title or author)
df.dropna(subset=['title', 'author'], inplace=True)

# STEP 4: Fill missing optional fields with blank
for col in ['edition', 'imprint', 'date']:
    df[col] = df[col].fillna('')

# STEP 5: Clean text: lowercase + strip spaces
for col in ['title', 'author', 'edition', 'imprint']:
    df[col] = df[col].astype(str).str.strip().str.lower()

# STEP 6: Format date as string
df['date'] = df['date'].astype(str).str.replace('.0', '', regex=False).str.strip()

# STEP 7: Create combined 'content' column
df['content'] = (
    df['title'] + ' ' +
    df['author'] + ' ' +
    df['edition'] + ' ' +
    df['imprint'] + ' ' +
    df['date']
)

# STEP 8: Remove duplicates based on item number, title, and author
df.drop_duplicates(subset=['item_no', 'title', 'author'], inplace=True)

# STEP 9: Keep only English entries
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

print("🌍 Filtering non-English rows...")
df = df[df['content'].apply(is_english)]

# STEP 10: Save the cleaned data
df.to_csv("cleaned_books.csv", index=False)
print("✅ Cleaned dataset saved as 'cleaned_books.csv'")


🌍 Filtering non-English rows...
